In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Loading the train and test dataset from torchvision Mnist dataset
train_data = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_size = 28 * 28
num_epochs = 5
learning_rate = 0.0001
batch_size = 64
num_classes = 10

In [4]:
train_loader = DataLoader(shuffle=True, dataset=train_data, batch_size=batch_size)
test_loader = DataLoader(shuffle=True, dataset=test_data, batch_size=batch_size)

In [5]:
# Neural Network with hidden layers
class NeuralNetwork(nn.Module):
  def __init__(self, input_size, num_classes) -> None:
    super().__init__()
    self.layer1 = nn.Linear(input_size, 200)
    self.layer2 = nn.ReLU()
    self.layer3 = nn.Linear(200, num_classes)
    self.input_size = input_size
  def forward(self, imgs):
    flatten_img = imgs.reshape(-1, self.input_size)
    out = self.layer1(flatten_img)
    out = self.layer2(out)
    out = self.layer3(out)
    return out

In [ ]:
# loading the model and evaluation
model = NeuralNetwork(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for i in range(num_epochs):
  for j, (t_images, t_labels) in enumerate(train_loader):
    t_labels = t_labels.to(device)
    output = model.forward(t_images)
    output = output.to(device)
    loss = criterion(output, t_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"epoch: {i}, loss: {loss}")

In [ ]:
total_lables = 0
predicted_labels = 0
with torch.no_grad():
  for t_images, t_labels in test_loader:
    output = model(t_images)
    _, predictions = torch.max(output, 1)
    total_lables += t_labels.size(0)
    predicted_labels += (t_labels == predictions).sum()
  print(f"Model Accuracy: {(predicted_labels/total_lables)*100}%")

In [ ]:
# save the model
torch.save(model.state_dict(), 'model.ckpt')